In [9]:
import requests
from bs4 import BeautifulSoup

In [2]:
urls = ['https://www.eldiarioar.com/politica/capitanich-no-descarto-gobierne-utilice-decretos-si-no-avanzan-sesiones-extraordinarias-congreso_1_9873916.html']

In [10]:
r = requests.get(urls[0])
print (r)
print (r.content)

<Response [200]>


In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

In [14]:
s = soup.find_all('p', class_='article-text')

In [16]:
for parrafo in s:
    print(parrafo.text)

SyntaxError: invalid non-printable character U+00A0 (1880082725.py, line 2)